In [1]:
import tensorflow as tf
import glob
import pandas as pd
import numpy as np

In [2]:
number = ['0','1','2','3','4','5','6','7','8','9']
alphabet = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
ALPHABET = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']
CAPTCHA_CHARSET = number+alphabet+ALPHABET   # 验证码字符集
TRAIN_DATA_DIR = './captcha_pic/' # 验证码数据集目录
TRAIN_DATASET_SIZE = 10 
BATCH_SIZE = 3
EPOCHS = 5



In [3]:
def read_pic():
    #读取图片数据
    #1.构造文件名队列
    #获取文件名队列
    file_names = glob.glob(TRAIN_DATA_DIR +'*.jpg')
    #print("file_names:\n",file_names)
    file_queue = tf.train.string_input_producer(file_names)
    #2.读取与解码
    reader = tf.WholeFileReader()
    filename,image = reader.read(file_queue)
    #解码
    decoded = tf.image.decode_jpeg(image)
    #更新形状，确定图片形状
    decoded.set_shape([60,160,3])
    #修改图片类型
    image_cast = tf.cast(decoded,tf.float32)
    #3. 批处理
    filename_batch,image_batch = tf.train.batch([filename,image_cast],batch_size=BATCH_SIZE,num_threads=1,capacity=200)
    return  filename_batch,image_batch

In [4]:
#解析csv文件
def parse_csv(charset=CAPTCHA_CHARSET):
    #读取文件
    labels=[]
    csv_data = pd.read_csv(TRAIN_DATA_DIR+'labels.csv',index_col="file_num")
    for label in csv_data["chars"]:
    #print(label)
        letter = []
        #根据字符生成对应数字
        for word in label:
            letter.append(charset.index((word)))
        labels.append(letter)
    csv_data["labels"] = labels
    return csv_data

In [34]:
#将一个样本值和目标值一一对应，通过文件名查表（csv_data）
def filename2label(filename,csv_data):
    labels = []
    for file_name in filename:
        file_num = "".join(list(filter(str.isdigit,str(file_name))))
        target = csv_data.loc[int(file_num),"labels"]
        #print(target)
        labels.append(target)
    return np.array(labels)

In [20]:
def create_weights(shape):
    return tf.Variable(initial_value=tf.random_normal(shape=shape,stddev=0.01))

In [28]:
#构建卷积神经网络
def create_model(x):
#x:[None,60,160,3]
    #1.第一个卷积层
    with tf.variable_scope("conv1"):
        #卷积层
        conv1_weights = create_weights(shape=[5,5,3,32])
        conv1_bias = create_weights(shape=[32])
        conv1_x = tf.nn.conv2d(input=x,filter=conv1_weights,strides=[1,1,1,1],padding="SAME")+conv1_bias
        #激活层
        relu1_x = tf.nn.relu(conv1_x)
        #池化层
        pool1_x = tf.nn.max_pool(value=relu1_x,ksize = [1,2,2,1],strides =[1,2,2,1],padding="SAME")
    #2.第二个卷积大层
    with tf.variable_scope("conv2"):
        #x:[None,30,80,32]
        #卷积层
        conv2_weights = create_weights(shape=[5,5,32,64])
        conv2_bias = create_weights(shape=[64])
        conv2_x = tf.nn.conv2d(input=pool1_x,filter=conv2_weights,strides=[1,1,1,1],padding="SAME")+conv2_bias
        #激活层
        relu2_x = tf.nn.relu(conv2_x)
        #池化层
        pool2_x = tf.nn.max_pool(value=relu2_x,ksize = [1,2,2,1],strides =[1,2,2,1],padding="SAME")
    #3.全连接层
    with tf.variable_scope("full_connection"):
        #x:[None,15,40,64]--x:[None,15*40*64]
        #[None,15*40*64]*[15*40*64,4*62] =[None,4*62]
        x_fc=tf.reshape(pool2_x,shape=[-1,15*40*64])
        weights_fc = create_weights(shape=[15*40*64,4*62])
        bias_fc = create_weights(shape=[4*62])
        y_predict = tf.matmul(x_fc,weights_fc)+bias_fc
    return y_predict

In [36]:
filename,image = read_pic()
csv_data = parse_csv(CAPTCHA_CHARSET)
#1.准备数据
x = tf.placeholder(tf.float32,shape =[None,60,160,3])
y_true = tf.placeholder(tf.float32,shape =[None,4*62])
#2.构建模型
y_predict = create_model(x)
#3.构建损失函数
loss_list = tf.nn.sigmoid_cross_entropy_with_logits(labels =y_true,logits=y_predict )
loss = tf.reduce_mean(loss_list)
#4.优化损失
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(loss)

#5.计算准确率
equal_list = tf.reduce_all(tf.equal(tf.argmax(tf.reshape(y_predict,shape=[-1,4,62]),axis=2),
tf.argmax(tf.reshape(y_true,shape=[-1,4,62]),axis=2)),axis=1)
accuracy = tf.reduce_mean(tf.cast(equal_list,tf.float32))
#初始化变量
init = tf.global_variables_initializer()
#开启会话```````````
with tf.Session() as sess:   
    #初始化变量
    sess.run(init)
    #开启线程
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess ,coord=coord)
    for i in range(EPOCHS):
        filename_value ,image_value = sess.run([filename ,image])
        #print("filename_value:\n",filename_value)
        #print("image_value:\n",image_value)
        labels = filename2label(filename_value,csv_data)
        #将标签值转换成one-hot
        labels_value = tf.reshape(tf.one_hot(labels,depth=62),[-1,4*62]).eval()
        #print(labels_value)
        _,error,accuray_value = sess.run([optimizer,loss,accuracy],feed_dict={x:image_value,y_true:labels_value})
        print("第%d次训练后损失为%f,准确率为%f"%(i+1,error,accuray_value))
    #回收线程
    coord.request_stop()
    coord.join(threads)
    sess.close()

filename_value:
 [b'./captcha_pic\\3.jpg' b'./captcha_pic\\5.jpg' b'./captcha_pic\\0.jpg']
image_value:
 [[[[241. 246. 252.]
   [241. 246. 252.]
   [241. 246. 252.]
   ...
   [243. 248. 252.]
   [241. 247. 247.]
   [241. 247. 245.]]

  [[241. 246. 252.]
   [241. 246. 252.]
   [241. 246. 252.]
   ...
   [238. 243. 247.]
   [239. 245. 245.]
   [243. 249. 247.]]

  [[241. 246. 252.]
   [241. 246. 252.]
   [241. 246. 252.]
   ...
   [238. 243. 247.]
   [240. 246. 246.]
   [243. 249. 247.]]

  ...

  [[242. 245. 255.]
   [240. 243. 255.]
   [239. 242. 255.]
   ...
   [241. 246. 252.]
   [241. 246. 252.]
   [241. 246. 252.]]

  [[240. 243. 255.]
   [241. 244. 255.]
   [242. 245. 255.]
   ...
   [241. 246. 252.]
   [241. 246. 252.]
   [241. 246. 252.]]

  [[238. 241. 255.]
   [239. 242. 255.]
   [241. 245. 255.]
   ...
   [241. 246. 252.]
   [241. 246. 252.]
   [241. 246. 252.]]]


 [[[240. 249. 246.]
   [240. 249. 246.]
   [240. 249. 246.]
   ...
   [240. 249. 246.]
   [239. 250. 246.]
   [2